In [1]:
!pip3 install -q -U transformers
!pip3 install -q -U datasets
!pip3 install -q -U bitsandbytes
!pip3 install -q -U peft
!pip3 install -q -U trl
!pip3 install -q -U accelerate
!pip3 install -q -U jinja2

In [2]:
import torch
from datasets import Dataset, load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline, TrainingArguments
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

tokenizer = AutoTokenizer.from_pretrained("google/gemma-1.1-2b-it")
model = AutoModelForCausalLM.from_pretrained(
    "google/gemma-1.1-2b-it",
    torch_dtype=torch.bfloat16,
    device_map="auto"
#    device_map={"":0}
)

tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/618 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

In [4]:
input_text = "Write me a poem about Machine Learning."
input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")

outputs = model.generate(**input_ids, max_new_tokens=50)
print(tokenizer.decode(outputs[0]))


<bos>Write me a poem about Machine Learning.

In circuits of code, a mind takes form,
A tapestry of algorithms, a digital storm.
Through layers of computation, data takes flight,
Learning patterns, insights in the night.

From images that speak, to algorithms that bind


In [5]:
text_gen = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens = 512
)

messages = [
    {"role": "user", 
     "content": "이순신 장군에 대해 설명해주세요."},
]
prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
print(prompt) 

Device set to use cuda:0


<bos><start_of_turn>user
이순신 장군에 대해 설명해주세요.<end_of_turn>
<start_of_turn>model



In [6]:
outputs = text_gen(
    prompt,
    max_new_tokens=4096,
    add_special_tokens=True
)
print(outputs[0]["generated_text"][len(prompt):]) 


**이순신 장군**은 조선의 제10대 장군으로, 1592년 조선을 건국한 것으로 알려져 있습니다.

**생애:**

* 1557년 3월 18일, 경기도 양평에서 출생했습니다.
* 조선의 13대 군주 조선왕의 막내로 출생했습니다.
* 1582년 조선을 건국했습니다.

**장군 활동:**

* 조선을 건국한 후, 이순신은 조선의 군사적, 정치적, 경제적 권력을 확대했습니다.
* 조선을 국경으로 넘어 일본, 한국, 일본 등 지역을 점령했습니다.
* 조선의 문화와 경제를 발전시켰습니다.

**사후:**

* 이순신은 조선의 첫 번째 대통령이되었습니다.
* 그의 영향은 조선의 역사에 지대한 영향을 미했습니다.
* 그의 이름은 조선의 국민의 이름으로 유명합니다.

**주요 업적:**

* 조선을 건국
* 조선의 군사적, 정치적, 경제적 권력 확대
* 조선의 문화와 경제 발전
* 조선의 국민들에게 영감을 줄 힘을 제공


In [7]:
from transformers import TextStreamer, GenerationConfig

streamer = TextStreamer(tokenizer)

def gen(prompt):
    generation_config = GenerationConfig(
        temperature = 1.31,
        max_new_tokens = 512,
        exponential_decay_length_penalty = (512, 1.03),
        eos_token_id = tokenizer.eos_token_id,
        repetition_penalty = 1.17,
        do_sample = True,
        top_k = 49,
        top_p = 0.14,
        min_length = 5,
        use_cache = True,
        return_dict_in_generate = True
    )
    gened = model.generate(
        **tokenizer(
            prompt,
            return_tensors='pt',
            return_token_type_ids=False
        ).to("cuda"),
        generation_config=generation_config,
        streamer=streamer
    )

In [8]:
messages = [
    {"role": "user", 
     "content": "이순신 장군에 대해 설명해주세요."},
]
prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True) 

gen(prompt)

`generation_config` default values have been modified to match model-specific defaults: {'pad_token_id': 0, 'bos_token_id': 2, 'transformers_version': '4.38.1'}. If this is not desired, please set these values explicitly.


<bos><bos><start_of_turn>user
이순신 장군에 대해 설명해주세요.<end_of_turn>
<start_of_turn>model
**이순신 장군**은 조선의 제10대 장군으로, 1592년부터 1608년까지 재위했습니다. 그는 조선의 군사적 지도와 정치적 권력을 확장하는 데 큰 기여를했습니다.

**생애:**

* 1573년 문종이 조선을 건국한 후, 이순신은 조선의 장수로서 활동했습니다.
* 1592년, 조선을 방문하여 조선의 군사적 강력함을 증명했습니다.
* 1598년, 조선을 북쪽으로 침략하여 동북 지역을 점령했습니다.

**지도:**

* 이순신은 조선의 군사적 지도자로, 조선의 전략적 위치를 개선하고 군사적 전략을 개선했습니다.
* 그는 조선의 군사적 우위를 높였고, 조선의 영역을 넓혔습니다.
* 그는 또한 조선의 정치적 권력을 확장하고, 조선의 통제권을 강화했습니다.

**기여:**

* 조선의 군사적 강력함을 높입니다.
* 조선의 영역을 넓힙니다.
* 조선의 정치적 권력을 확장합니다.
* 조선의 역사에 중요한 역할을 합니다.

**영향:**

* 이순신은 조선의 역사에 큰 영향을 미쳤습니다.
* 그의 지도는 조선의 군사적 강력함과 정치적 권력을 높입니다.
* 그의 이름은 조선의 장군으로 유명하며, 그의 이름은 조선의 국민들에게 전통적인 이미지를 제공합니다.<eos>


In [9]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TextIteratorStreamer
from threading import Thread

def gen(prompt):
    inputs = tokenizer([prompt], return_tensors="pt").to("cuda")
    streamer = TextIteratorStreamer(tokenizer)

    generation_kwargs = dict(inputs, streamer=streamer, max_new_tokens=256)
    thread = Thread(target=model.generate, kwargs=generation_kwargs)
    thread.start()
    return streamer

In [10]:
messages = [
    {"role": "user", 
     "content": "이순신 장군에 대해 설명해주세요."},
]
prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True) 

streamer = gen(prompt)
is_first_chunk = True
for text in streamer:
    if is_first_chunk:
        is_first_chunk = False
        continue
    print(text, end='')


**이순신 장군**은 조선의 제10대 장군으로, 1592년 조선을 건국한 것으로 알려져 있습니다.

**생애:**

* 1557년 3월 18일, 경기도 양평에서 출생했습니다.
* 조선의 13대 군주 조선왕의 막내로 출생했습니다.
* 1582년 조선을 건국했습니다.

**장군 활동:**

* 조선을 건국한 후, 이순신은 조선의 군사적, 정치적, 경제적 권력을 확대했습니다.
* 조선을 국경으로 넘어 일본, 한국, 일본 등 지역을 점령했습니다.
* 조선의 문화와 경제를 발전시켰습니다.

**사후:**

* 이순신은 조선의 첫 번째 대통령이되었습니다.
* 그의 영향은 조선의 역사에 지대한 영향을 미했습니다.
* 그의 이름은 조선의 국민의 이름으로 유명합니다.

**주요 업적:**

* 조